# **CRIANDO DAS TABELAS DIMENSÕES,FATOS E MÉTRICAS** #

**CRIANDO AS TABELAS DIMENSÕES**

1. As tabelas dimensões são criadas relacionando a tabela execucao_financeira_despesa------
2. Na tabela execucao_financeira_despesa, contém dados NULOS e NEGATIVOS e valores fora dos padrões (minusculo, maiusculo) e duplicados----
3. Estamos fazendo o tratamento dos dados na criação das tabelas dim e fato, porque a padronização facilita a filtragem, agrupamento e ordenação dos dados,  facilitando a integração e a interoperabilidade.

> Tabela de Dimensão de Credor = dim_credor

In [ ]:
----Criação----
CREATE TABLE data_warehouse.dim_credor AS
SELECT DISTINCT
     COALESCE(cod_credor, 'N/A') AS codigo_credor
    ,TRIM(UPPER(COALESCE(dsc_nome_credor, 'Não Especificado'))) AS nome_credor
FROM public.execucao_financeira_despesa
WHERE 1 = 1;

---- Consulta-----
SELECT *
FROM data_warehouse.dim_credor;

**OBS** *AQUI NO CREDOR ACONTECE A MESMA SITUAÇÃO DO ORGÃO CODIGO DO CREDOR UNICO, MAIS NA DESCRIÇÃO FOI INCLUSO FORMAS DE ESCRITA DIFERENTES MAIS QUE SÃO O MESMO CREDOR, PARA RESOLVER ISSO DEVE TER UMA PADRONIZAÇÃO DOS NOMES FAZENDO UM UPDATE NA TABELA DIM_ORGAO.*

> Tabela de Dimensão de Órgão = dim_orgão

In [ ]:
----Criação----
CREATE TABLE data_warehouse.dim_orgao AS
SELECT DISTINCT
     COALESCE(codigo_orgao, 'N/A') AS codigo_orgao
    ,TRIM(UPPER(COALESCE(dsc_orgao, 'Não Especificado'))) AS nome_orgao
FROM public.execucao_financeira_despesa
WHERE 1 = 1;

---- Consulta-----
SELECT *
FROM data_warehouse.dim_orgao;


Foi analisado que o código do orgão são iguais mesmo que o nome do orgão tenha uma diferença de escrita, então nas analises deve ser considerado o codigo do orgao, aqui nesse caso estamos fazendo um UPDATE apenas em situações em que existe o código do orgao mais o campo estava nulo, alteramos para não especificado; e quando o nome estava com duas escritas diferentes e o código era o mesmo, vamos deixar padrão e agora vamos alterar para o preenchimeto correto do campo.

In [ ]:
-----Usamos essa query para analisar as diferenças----

SELECT codigo_orgao, nome_orgao
FROM data_warehouse.dim_orgao
WHERE codigo_orgao IN (
    SELECT codigo_orgao
    FROM data_warehouse.dim_orgao
    GROUP BY codigo_orgao
    HAVING COUNT(*) > 1
);

SELECT *
FROM data_warehouse.dim_credor;

----UPDATE------

UPDATE data_warehouse.dim_orgao
SET nome_orgao = 
    CASE 
        WHEN codigo_orgao = '561001' THEN 'FUNDO DE INVESTIMENTOS DO MICROCREDITO PRODUTIVO DO CEARA'
        WHEN codigo_orgao = '561101' THEN 'FUNDO DE DEFESA AGROPECUARIA DO ESTADO DO CEARA'
        WHEN codigo_orgao = '010101' THEN 'FUNDO DE PREVIDENCIA PARLAMENTAR DA ASSEMBLEIA LEGISLATIVA DO CEARÁ'
        WHEN codigo_orgao = '310601' THEN 'FUNDACAO NUCLEO DE TECNOLOGIA E QUALIDADE INDUSTRIAL DO CEARA'
        ELSE nome_orgao
    END;

-----EXCLUINDO REGISTROS DUPLICADOS------

DELETE FROM data_warehouse.dim_orgao
WHERE codigo_orgao IN (
    SELECT codigo_orgao
    FROM data_warehouse.dim_orgao
    GROUP BY codigo_orgao
    HAVING COUNT(*) > 1
) AND ctid NOT IN (
    SELECT min(ctid)
    FROM data_warehouse.dim_orgao
    GROUP BY codigo_orgao
    HAVING COUNT(*) > 1
);

> Tabela de Dimensão de Item = dim_item

In [ ]:
----Criação----
CREATE TABLE data_warehouse.dim_item AS
SELECT DISTINCT
     COALESCE(cod_item, 'N/A') AS codigo_item
    ,TRIM(UPPER(COALESCE(dsc_item, 'Não Especificado'))) AS descricao_item
FROM public.execucao_financeira_despesa
WHERE 1 = 1;

> Tabela de Dimensão de Item = dim_elemento

In [ ]:
----Criação----
CREATE TABLE data_warehouse.dim_elemento AS
SELECT DISTINCT
     COALESCE(cod_item_elemento, 'N/A') AS codigo_item_elemento
    ,TRIM(UPPER(COALESCE(dsc_item_elemento, 'Não Especificado'))) AS descricao_item_elemento
FROM public.execucao_financeira_despesa
WHERE 1 = 1;

---- Consulta-----
SELECT *
FROM data_warehouse.dim_elemento;

> Tabela de Dimensão de Item = dim_categoria

In [ ]:
----Criação----
CREATE TABLE data_warehouse.dim_categoria AS
SELECT DISTINCT
     COALESCE(cod_item_categoria, 'N/A') AS codigo_item_categoria
    ,TRIM(UPPER(COALESCE(dsc_item_categoria, 'Não Especificado'))) AS descricao_item_categoria
FROM public.execucao_financeira_despesa
WHERE 1 = 1;

---- Consulta-----
SELECT *
FROM data_warehouse.dim_categoria;

--------PADRONIZANDO-----
SELECT codigo_item_categoria, descricao_item_categoria
FROM data_warehouse.dim_categoria
WHERE codigo_item_categoria IN (
    SELECT codigo_item_categoria
    FROM data_warehouse.dim_categoria
    GROUP BY codigo_item_categoria
    HAVING COUNT(*) > 1
)
ORDER BY codigo_item_categoria ASC;

-------UPDATE-------
UPDATE data_warehouse.dim_categoria
SET descricao_item_categoria = 
    CASE 
        WHEN codigo_item_categoria = '3' THEN 'DESPESA CORRENTE'
        WHEN codigo_item_categoria = '4' THEN 'DESPESA DE CAPITAL'
        ELSE descricao_item_categoria

    END;

------EXCLUINDO REGISTROS DUPLICADOS----

DELETE FROM data_warehouse.dim_categoria
WHERE codigo_item_categoria IN (
    SELECT codigo_item_categoria
    FROM data_warehouse.dim_categoria
    GROUP BY codigo_item_categoria
    HAVING COUNT(*) > 1
) AND ctid NOT IN (
    SELECT min(ctid)
    FROM data_warehouse.dim_categoria
    GROUP BY codigo_item_categoria
    HAVING COUNT(*) > 1
);

> Tabela de Dimensão de Item = dim_grupo

In [ ]:
----Criação----
CREATE TABLE data_warehouse.dim_grupo AS
SELECT DISTINCT
     COALESCE(cod_item_grupo, 'N/A') AS codigo_item_grupo
     ,TRIM(UPPER(COALESCE(dsc_item_grupo, 'Não Especificado'))) AS descricao_item_grupo
FROM public.execucao_financeira_despesa
WHERE 1 = 1;

---- Consulta-----
SELECT *
FROM data_warehouse.dim_grupo;


> Tabela de Dimensão de Subfunção = dim_subfuncao

In [ ]:
----Criação----
CREATE TABLE data_warehouse.dim_subfuncao AS
SELECT DISTINCT
    COALESCE(cod_subfuncao, 'N/A') AS codigo_subfuncao
   ,TRIM(UPPER(COALESCE(dsc_subfuncao, 'Não Especificado'))) AS descricao_subfuncao
FROM public.execucao_financeira_despesa
WHERE 1 = 1;

---- Consulta-----
SELECT *
FROM data_warehouse.dim_subfuncao;

> Tabela de Dimensão de Programa = dim_programa

In [ ]:
----Criação----
CREATE TABLE data_warehouse.dim_programa AS
SELECT DISTINCT
    COALESCE(cod_programa, 'N/A') AS codigo_programa
   ,TRIM(UPPER(COALESCE(dsc_programa, 'Não Especificado'))) AS descricao_programa
FROM public.execucao_financeira_despesa
WHERE 1 = 1;

---- Consulta-----
SELECT *
FROM data_warehouse.dim_programa;

> Tabela de Dimensão de Modalidade de Licitação = dim_modalidade_licitacao

In [ ]:
----Criação----
CREATE TABLE data_warehouse.dim_modalidade_licitacao AS
SELECT DISTINCT
    COALESCE(cod_item_modalidade, 'N/A') AS codigo_item_modalidade
   ,TRIM(UPPER(COALESCE(dsc_modalidade_licitacao, 'Não Especificado'))) AS descricao_modalidade_licitacao
FROM public.execucao_financeira_despesa
WHERE 1 = 1;

---- Consulta-----
SELECT *
FROM data_warehouse.dim_modalidade_licitacao;

> Tabela de Dimensão de Função = dim_funcao

In [ ]:
----Criação----
CREATE TABLE data_warehouse.dim_funcao AS
SELECT DISTINCT
     COALESCE(cod_funcao, 'N/A') AS codigo_funcao
    ,TRIM(UPPER(COALESCE(dsc_funcao, 'Não Especificado'))) AS descricao_funcao
FROM public.execucao_financeira_despesa
WHERE 1 = 1;

---- Consulta-----
SELECT *
FROM data_warehouse.dim_funcao;

> Tabela de Dimensão de Fonte = dim_fonte

In [ ]:
----Criação----
CREATE TABLE data_warehouse.dim_fonte AS
SELECT DISTINCT
   COALESCE(cod_fonte, 'N/A') AS codigo_fonte
  ,TRIM(UPPER(COALESCE(dsc_fonte, 'Não Especificado'))) AS descricao_fonte
FROM public.execucao_financeira_despesa
WHERE 1 = 1;

---- Consulta-----
SELECT *
FROM data_warehouse.dim_fonte;

> Tabela de Dimensão de Número de Processo = dim_numero_processo

In [ ]:
----Criação----
CREATE TABLE data_warehouse.dim_numero_processo AS
SELECT DISTINCT
   COALESCE(cod_np, 'N/A') AS codigo_numero_processo
  ,TRIM(UPPER(COALESCE(num_ano_np, 'Não Especificado'))) AS ano_processo
FROM public.execucao_financeira_despesa
WHERE 1 = 1;

---- Consulta-----
SELECT *
FROM data_warehouse.dim_numero_processo;

> **Passo a passo de como criar uma TABELA DIMENSÃO**

1. Primeiro usamos a cláusula "CREATE TABLE".
2. A tabela está sendo preenchida com os resultados da consulta especificada após o "AS".
3. A consulta começa com a cláusula "SELECT", indicando as colunas a serem selecionadas.
4. A primeira coluna selecionada é "COALESCE('coluna', 'N/A') AS 'coluna' ". Aqui, a função "COALESCE" é usada para retornar o valor da coluna. Se o valor for nulo, será retornado 'N/A' como um valor alternativo. 
5. A coluna resultante é renomeada como "coluna".
6. A próxima coluna é "TRIM(UPPER(COALESCE('coluna', 'Não Especificado')) AS coluna". Aqui, a função "COALESCE" é usada novamente para retornar o valor da coluna "num_ano_np". Se o valor for nulo, será retornado 'Não Especificado' como um valor alternativo. Em seguida, a função a função "TRIM" é aplicada para garantir que os espaços em branco sejam eliminados, "UPPER" é aplicada para converter o valor para letras maiúsculas. 
7. A coluna resultante é renomeada como "coluna".
8. A cláusula "FROM public.execucao_financeira_despesa" especifica a tabela da qual queremos obter os dados.
9. A cláusula "WHERE 1 = 1" é usada para adicionar uma condição de filtro. Neste caso, não há condição específica, então todos os dados serão selecionados.
10. Usamos A FUNÇÃO COALESCE ela é útil quando se deseja fornecer um valor alternativo ou substituir valores nulos por algum valor padrão em uma expressão. 
11. Isso é especialmente útil em consultas e transformações de dados, onde é importante lidar com valores nulos de forma adequada.
12. No tratamento de dados, jamais podemos alterar o banco de dados de produção (padrão), por esse motivo essa foi a melhor forma que encontramos de tratar os campos nulos na tabela execucao_financeira_despesa, sem alterar os valores contidos nela.
13. Aplicamos o resultado 'N/A, nas colunas correspondentes a códigos, caso estivessem NOT NULL e aplicamos 'Não Especificado' nas colunas que estivessem descrição; E para aproveitar a alteração das colunas NOT NULL, nos campos com TEXT, aplicamos também a função UPPER, que já altera as strings para MAIÚSCULA.
14. Criamos as dimensões com os Selects padrões, mesmo que não estivesse um campo nulo ou string fora do padrão, para que na inserção de um novo dado, caso ocorra essa situação a dimensão já consegue ficar dentro do padrão.
15. Fizemos algumas padronizações nas tabelas dim_categoria e dim_orgão, como a atualização das descrições com mesmo código e um delete nas informações duplicadas.

NOT IN: A cláusula NOT IN é usada para verificar se um valor não está presente em uma lista de valores. No comando fornecido, ela é usada para verificar se o valor da coluna ctid não está presente na subconsulta.

min(): A função min() é usada para retornar o valor mínimo de uma coluna. No comando fornecido, é usada na subconsulta para obter o valor mínimo de ctid para cada grupo de codigo_orgao.

**OBS**: A query de Delete, conseguimos buscar informações de como fazer na internet, foruns e chats.


> Tabela de Dimensão de DATA = dim_data

In [ ]:
---------função para alterar os nomes em inglês para português------

CREATE OR REPLACE FUNCTION mes_nome_pt(data date)
  RETURNS TEXT AS $$
DECLARE
  meses TEXT[] := ARRAY['Janeiro', 'Fevereiro', 'Marco', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro'];
BEGIN
  RETURN meses[EXTRACT(MONTH FROM data)];
END;
$$ LANGUAGE plpgsql;

---------depois criar a tabela data---------
 
CREATE TABLE data_warehouse.dim_data AS
SELECT DISTINCT 
       to_date(num_ano::text, 'YYYY')                          AS Data
     , EXTRACT(DAY FROM CURRENT_DATE)                          AS Dia
     , EXTRACT(WEEK FROM CURRENT_DATE)                         AS Semana
     , EXTRACT(MONTH FROM to_date(num_ano::text, 'YYYY'))      AS Mês
     , to_char(to_date(num_ano::text, 'YYYY'), 'Month')        AS MêsNome
     , CEILING(date_part('month', to_date(num_ano::text, 'YYYY')) / 2.0)::integer AS bimestre
     , CEILING(date_part('month', to_date(num_ano::text, 'YYYY')) / 3.0)::integer AS trimestre
     , CEILING(date_part('month', to_date(num_ano::text, 'YYYY')) / 6.0)::integer AS semestre
     , EXTRACT(YEAR FROM to_date(num_ano::text, 'YYYY'))       AS Ano
  FROM public.execucao_financeira_despesa
 WHERE 1 = 1
 ORDER BY 1;

> **PASSO A PASSO de como criar a DIM_DATA**

1. Primeiro executar a função; a função foi feita para alterar na dim data, os meses que trazem resultado em Inglês para português enfatizando que utilizamos a mesma dim_data que trabalhamos em sala de aula, não criamos a função exatamente.
2. Primeiro passamos a cláusula "CREATE TABLE", no schema data_warehouse com o nome da tabela dim_data.
3. A tabela está sendo preenchida com os resultados da consulta especificada após o "AS".
4. A consulta começa com a cláusula "SELECT DISTINCT", indicando que queremos selecionar apenas valores únicos.
5. A primeira coluna selecionada é "to_date(num_ano::text, 'YYYY') AS Data". Aqui, a função "to_date" é usada para converter o valor da coluna "num_ano" em um formato de data, especificado como 'YYYY' (ano de quatro dígitos); O resultado é renomeado como "Data".
6. A próxima coluna é "EXTRACT(DAY FROM CURRENT_DATE) AS Dia". Aqui, a função "EXTRACT" é usada para extrair o dia atual do "CURRENT_DATE" (data atual).
7. Em seguida, temos "EXTRACT(WEEK FROM CURRENT_DATE) AS Semana". A função "EXTRACT" é usada para extrair o número da semana atual do "CURRENT_DATE".
8. A próxima coluna é "EXTRACT(MONTH FROM to_date(num_ano::text, 'YYYY')) AS Mês". Aqui, a função "EXTRACT" é usada para extrair o mês da data convertida da coluna "num_ano".
9. A coluna "to_char(to_date(num_ano::text, 'YYYY'), 'Month') AS MêsNome" usa a função "to_char" para converter a data em formato 
10. de mês por extenso (ex: "January", "February").
11. As colunas "bimestre", "trimestre" e "semestre" são calculadas usando a função "CEILING" e "date_part". 
12. Elas dividem o mês extraído anteriormente por 2, 3 e 6, respectivamente, e arredondam para cima.
13. A coluna "EXTRACT(YEAR FROM to_date(num_ano::text, 'YYYY')) AS Ano" extrai o ano da data convertida da coluna "num_ano".
14. A cláusula "FROM public.execucao_financeira_despesa" especifica a tabela da qual queremos obter os dados.

# **CRIANDO A TABELA FATO** #

In [ ]:
-- Tabela de Fato Empenho = fato_empenho

CREATE TABLE data_warehouse.fato_empenho AS
SELECT DISTINCT ON (codigo_empenho, ano_cadastro, codigo_credor, codigo_orgao)
    COALESCE(cod_ne, 'N/A') AS codigo_empenho,
    num_ano AS ano_cadastro,
    COALESCE(cod_credor, 'N/A') AS codigo_credor,
    COALESCE(codigo_orgao,'N/A') AS codigo_orgao,
    COALESCE(cod_np, 'N/A') AS codigo_numero_processo,
    COALESCE(cod_fonte, 'N/A') AS codigo_fonte,
    COALESCE(cod_funcao, 'N/A') AS codigo_funcao,
    COALESCE(cod_item_modalidade, 'N/A') AS codigo_item_modalidade,
    COALESCE(cod_programa, 'N/A') AS codigo_programa,
    COALESCE(cod_subfuncao, 'N/A') AS codigo_subfuncao,
    COALESCE(cod_item_grupo, 'N/A') AS codigo_item_grupo,
    COALESCE(cod_item_categoria, 'N/A') AS codigo_item_categoria,
    COALESCE(cod_item_elemento, 'N/A') AS codigo_item_elemento,
    COALESCE(cod_item, 'N/A') AS codigo_item,
    vlr_empenho AS valor_empenho,
    CASE
        WHEN valor_pago = vlr_empenho THEN valor_pago
        ELSE COALESCE(vlr_liquidado, 0.00)
    END AS valor_liquidado,
    COALESCE(valor_pago, 0.00) AS valor_pago,
    COALESCE(vlr_empenho - valor_pago, 0.00) AS valor_resto_a_pagar,
    dth_empenho AS data_empenho,
    dth_liquidacao AS data_liquidacao,
    dth_pagamento AS data_pagamento,
    dth_processamento AS data_processamento,
    COALESCE(num_sic, 'N/A') AS numero_identificacao_financeiro,
    CONCAT(num_ano::text, COALESCE(cod_ne, 'N/A'), COALESCE(codigo_orgao, 'N/A'), COALESCE(cod_credor, 'N/A')) AS id_fato_empenho
FROM public.execucao_financeira_despesa
WHERE 1 = 1;


-- Fazendo alteração na coluna data_liquidacao, para preencher com valores obedecendo a condição----

UPDATE data_warehouse.fato_empenho
SET data_liquidacao = data_pagamento
WHERE valor_liquidado > 0.00;

-- Fazendo alteração na coluna valor_empenho, para preencher apenas com os valores positivos----

UPDATE data_warehouse.fato_empenho
SET valor_empenho = 0.00
WHERE valor_empenho < 0.00;

-- Fazendo alteração na coluna valor_resto_a_pagar, para preencher com os valores do fato empenho, que tenha valor liquidado e pago igual a 0.
---Ele vai levar o mesmo valor do empenho para preencher o campo valor_resto_a_pagar.

UPDATE data_warehouse.fato_empenho
SET valor_resto_a_pagar = valor_empenho
WHERE valor_liquidado = 0.00 AND valor_pago = 0.00;

--Fazendo alteração na valor_resto_a_pagar, considerando a lógica de que quando o valor quando o (valor empenho - valor pagar) for um resultado negativo, 
--preencher o campo resto a pagar com 0.00 


> Passo a Passo da criação da TABELA FATO EMPENHO

A tabela **fato_empenho** foi criada a partir de uma consulta na tabela **execucao_financeira_despesa** com algumas transformações nos dados. 

**Criação da tabela:** A instrução CREATE TABLE data_warehouse.fato_empenho AS cria a tabela fato_empenho no esquema data_warehouse com base no resultado da consulta.

**Distinct On:** A cláusula DISTINCT ON é usada para retornar apenas um registro distinto para cada combinação única dos campos especificados. No caso do comando fornecido, os campos codigo_empenho, ano_cadastro, codigo_credor e codigo_orgao são utilizados para determinar a distinção.

**COALESCE:** A função COALESCE é usada para substituir valores nulos por um valor alternativo. No comando fornecido, as colunas cod_ne, cod_credor, codigo_orgao, cod_np, cod_fonte, cod_funcao, cod_item_modalidade, cod_programa, cod_subfuncao, cod_item_grupo, cod_item_categoria, cod_item_elemento e cod_item são tratadas com COALESCE para substituir valores nulos por 'N/A'.

**Alias e seleção de colunas:** A instrução AS é usada para fornecer aliases (apelidos) para as colunas selecionadas na consulta. Os aliases são usados para atribuir nomes mais descritivos às colunas resultantes. As colunas selecionadas incluem as transformações descritas e também algumas colunas que são copiadas diretamente da tabela original.

**CASE:** A cláusula CASE é usada para realizar avaliações condicionais e retornar valores com base nas condições especificadas. No comando fornecido, é utilizado um CASE para determinar o valor da coluna valor_liquidado. Se o valor da coluna valor_pago for igual a vlr_empenho, então o valor de valor_pago é retornado. Caso contrário, é retornado o valor de vlr_liquidado (se não for nulo) ou 0.00.

**WHERE 1=1:** Essa condição WHERE 1=1 é um truque comum usado para permitir a adição de cláusulas de filtro adicionais de forma mais fácil. Neste caso, essa condição não afeta o resultado da consulta.

**CONCAT:** A função CONCAT é utilizada para concatenar valores de diferentes colunas ou strings em uma única string. Aqui contatenamos os códigos das tabelas, dim_ano, dim_codigo, dim_orgao e dim_credor. A criação do id_empenho é para retirar as informações duplicadas.

>Após a criação da tabela, são executadas algumas instruções UPDATE para fazer alterações nos dados já inseridos na tabela fato_empenho.

**Atualização da coluna data_liquidacao:** A instrução **UPDATE** modifica os valores da coluna data_liquidacao para serem iguais aos valores da coluna data_pagamento quando o valor da coluna valor_liquidado é maior que 0.00.

**Atualização da coluna valor_empenho:** A instrução UPDATE modifica os valores da coluna valor_empenho para serem 0.00 quando o valor original é menor que 0.00.

**Atualização da coluna valor_resto_a_pagar:** A instrução UPDATE modifica os valores da coluna valor_resto_a_pagar para serem iguais aos valores da coluna valor_empenho quando tanto o valor da coluna valor_liquidado quanto o valor da coluna valor_pago são iguais a 0.00.

**Atualização da coluna valor_resto_a_pagar com lógica adicional:** A instrução UPDATE modifica os valores da coluna valor_resto_a_pagar para serem 0.00 quando a diferença entre valor_empenho e valor_pago for negativa, ou seja, quando (valor_empenho - valor_pago) é menor que 0.00.